In [1]:
!kaggle datasets download -d vivek468/beginner-chest-xray-image-classification

Dataset URL: https://www.kaggle.com/datasets/vivek468/beginner-chest-xray-image-classification
License(s): other
100% 2.08G/2.08G [01:43<00:00, 24.5MB/s]
100% 2.08G/2.08G [01:43<00:00, 21.5MB/s]


In [2]:
import zipfile

# ZIP dosyasının yolu
zip_file_path = '/content/beginner-chest-xray-image-classification.zip'

# Çıkartılacak hedef klasör
extracted_folder_path = '/content/'

# ZIP dosyasını aç
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [11]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Veri setinin dizini
data_dir = "/content/chest_xray"

# Veri ön işleme ve artırma
data_transforms = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet mean ve std
])

# Veri yükleyicileri oluşturma
train_dataset = datasets.ImageFolder(root=data_dir + '/train', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = datasets.ImageFolder(root=data_dir + '/test', transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [12]:
import torch.nn as nn
import torchvision.models as models

# VGG19 modelini yükleme
inceptionv3 = models.inception_v3(pretrained=True)

# Son fully connected layer'ı değiştirme
num_ftrs = inceptionv3.fc.in_features
inceptionv3.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Eğitim için CUDA kullanılabilirse GPU'ya taşıma ve DataParallel kullanarak sarmalama
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inceptionv3 = nn.DataParallel(inceptionv3)  # DataParallel ile modeli sarmalama
inceptionv3 = inceptionv3.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 80.2MB/s] 


In [16]:
import torch.optim as optim
import torch.nn.functional as F

from torch.optim.lr_scheduler import StepLR

optimizer = optim.SGD(inceptionv3.parameters(), lr=0.001, momentum=0.9)
#scheduler = StepLR(optimizer, step_size=3, gamma=0.1)
criterion = nn.CrossEntropyLoss()

from tqdm import tqdm

epochs = 1
for epoch in range(epochs):
    running_loss = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", unit="batch", colour='green', leave=True)
    for batch_idx, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        # Gradientleri sıfırlama
        optimizer.zero_grad()

        #Forward pass
        outputs, aux_outputs = inceptionv3(inputs)  # InceptionV3 modelinden hem çıkışlar hem de yardımcı çıkışlar alınır
        loss1 = criterion(outputs, labels)  # Ana çıkışlar için CrossEntropyLoss hesaplanır
        loss2 = criterion(aux_outputs, labels)  # Yardımcı çıkışlar için de CrossEntropyLoss hesaplanır
        loss = loss1 + 0.4 * loss2  # Ana çıkışlara daha fazla ağırlık vermek için yardımcı çıkışlar için kayıp 0.4 ile çarpılır


        # Backward pass ve optimizasyon
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        progress_bar.set_postfix({'Loss': loss.item()})

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/5: 100%|██████████| 229/229 [04:09<00:00,  1.09s/batch, Loss=0.00567]


Epoch 1/5, Loss: 0.0377


Epoch 2/5: 100%|██████████| 229/229 [04:10<00:00,  1.09s/batch, Loss=0.0632]


Epoch 2/5, Loss: 0.0258


Epoch 3/5: 100%|██████████| 229/229 [04:10<00:00,  1.09s/batch, Loss=0.0136]


Epoch 3/5, Loss: 0.0124


Epoch 4/5: 100%|██████████| 229/229 [04:06<00:00,  1.08s/batch, Loss=0.0023]


Epoch 4/5, Loss: 0.0062


Epoch 5/5: 100%|██████████| 229/229 [04:06<00:00,  1.08s/batch, Loss=0.00152]

Epoch 5/5, Loss: 0.0040


In [18]:
correct = 0
total = 0
TP = 0
FP = 0
FN = 0
TN = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs, _ = inceptionv3(inputs)  # InceptionV3 modelinden çıktılar alınır
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Kesinlik ve Duyarlılık için TP, FP ve FN değerlerini hesapla
        TP += ((predicted == labels) & (labels == 1)).sum().item()
        FP += ((predicted != labels) & (labels == 0)).sum().item()
        FN += ((predicted != labels) & (labels == 1)).sum().item()
        TN += ((predicted == labels) & (labels == 0)).sum().item()

accuracy = correct / total

# Kesinlik (Precision) hesapla
precision = TP / (TP + FP)

# Duyarlılık (Recall) hesapla
recall = TP / (TP + FN)

# F1-Skoru hesapla
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

print(f"TP: {TP:.4f}")
print(f"FP: {FP:.4f}")
print(f"FN: {FN:.4f}")
print(f"TN: {TN:.4f}")


Test Accuracy: 0.5984
Precision: 0.7009
Recall: 0.6192
F1-Score: 0.6575
TP: 239.0000
FP: 102.0000
FN: 147.0000
TN: 132.0000


In [7]:
torch.cuda.empty_cache()